# IMA205 Estienne GOIGOUX

<h3>Operational notebook</h3>
<br>
This notebook only runs the models and computes the predictions. You have to fill the IMAGE_DIR_TRAIN, panda_path_train, IMAGE_DIR_TEST, panda_path_test and run the main to get the predictions.

The scientific approach has been to implement steps and come back to precedent to better the performance and this notebook does not represent the whole work or code. The steps are described in each notebook :
<br>
* Step 1 : data analysis, https://www.kaggle.com/code/estienneggx/1-dataset-analysis
* Step 2 : Pre-processing and features extraction, https://www.kaggle.com/code/estienneggx/2-pre-processing-and-features-extraction
* Step 3 : Building models, https://www.kaggle.com/code/estienneggx/3a-random-forest-on-tabular-data and https://www.kaggle.com/code/estienneggx/3b-cnn-on-images
* Step 4 : Blending predictions https://www.kaggle.com/code/estienneggx/4-blending-models-and-final-predictions

In [18]:
import os
import cv2
import skimage
import sklearn
import scipy
import matplotlib
import PIL
import tensorflow as tf

from skimage.measure import label, regionprops, regionprops_table
from skimage.filters import sobel
from skimage import morphology, measure, segmentation, exposure, color
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix
from scipy.ndimage import uniform_filter

from PIL import Image

from os import listdir
from os.path import isfile, join
from tqdm import tqdm

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling as pdp
import matplotlib.pyplot as plt
import seaborn as sns


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.compat.v1.logging import INFO, set_verbosity




%matplotlib inline

print('PIL version : 9.0.1',
'cv2 version : 4.5.4',
'skimage version : 0.19.2',
'sklearn version : 1.0.2',
'scipy version : 1.7.3',
'pandas version : 1.3.5',
'matplotlib version : 3.5.1',
'seaborn version : 0.11.2',
'tensorflow version : 2.6.3',
'xgboost version : 1.6.0')

PIL version : 9.0.1 cv2 version : 4.5.4 skimage version : 0.19.2 sklearn version : 1.0.2 scipy version : 1.7.3 pandas version : 1.3.5 matplotlib version : 3.5.1 seaborn version : 0.11.2 tensorflow version : 2.6.3 xgboost version : 1.6.0


In [19]:
weights_dict = {
    1 : 0.7005531, 
    2 : 0.24592265, 
    3 : 0.95261733, 
    4 : 3.64804147, 
    5 : 1.20674543, 
    6 : 13.19375, 
    7 : 12.56547619, 
    8 : 5.04219745
}

weights = [ 0.7005531, 0.24592265, 0.95261733, 3.64804147, 1.20674543, 13.19375, 12.56547619, 5.04219745]


def rgb2yiq(imgArray):
    '''
    Simple mathematical operation for Adaptative thresholding implementation
    '''
    assert (imgArray.shape[2] == 3)
    l = imgArray.shape[0]
    L = imgArray.shape[1]
    
    YIQ = np.zeros((l, L, 3))
    YIQ[:,:,0] = (0.299 * imgArray[:,:,0] + 0.587 * imgArray[:,:,1] + 0.114 * imgArray[:,:,2])
    #YIQ[:,:,1] = 0.596 * imgArray[:,:,0] - 0.274 * imgArray[:,:,1] - 0.322 * imgArray[:,:,2]
    #YIQ[:,:,2] = 0.211 * imgArray[:,:,0] - 0.523 * imgArray[:,:,1] + 0.312 * imgArray[:,:,2]
    return np.asarray(YIQ)


def invertPixelsMinusMean(blackAndWhite):
    '''
    Simple mathematical operation for Adaptative thresholding implementation
    '''
    inverted = (blackAndWhite.max() - blackAndWhite)
    inverted = inverted - inverted.mean()
    inverted[inverted < 0] = 0
    
    return inverted

def averageFilter(img, filterSize):
    if (filterSize == 0):
        return img
    else:
        kernel = np.ones((filterSize,filterSize),np.float32)/(filterSize**2)
        dst = cv2.filter2D(img,-1,kernel)
    return dst

# https://github.com/AndersDHenriksen/SanityChecker/blob/master/AllChecks.py
def bwareafilt(mask, n=1, area_range=(0, np.inf)):
    '''
    Extracts the largest white area of binary img
    '''
    # For openCV > 3.0 this can be changed to: areas_num, labels = cv2.connectedComponents(mask.astype(np.uint8))
    labels = measure.label(mask.astype('uint8'), background=0)
    area_idx = np.arange(1, np.max(labels) + 1)
    areas = np.array([np.sum(labels == i) for i in area_idx])
    inside_range_idx = np.logical_and(areas >= area_range[0], areas <= area_range[1])
    area_idx = area_idx[inside_range_idx]
    areas = areas[inside_range_idx]
    keep_idx = area_idx[np.argsort(areas)[::-1][0:n]]
    kept_areas = areas[np.argsort(areas)[::-1][0:n]]
    
    if np.size(kept_areas) == 0:
        kept_areas = np.array([0])
    if n == 1:
        kept_areas = kept_areas[0]
    kept_mask = np.isin(labels, keep_idx)
    
    return kept_mask


# Gives basic stats of the RGB channels of an image
def getRGBstats(imPath):
    '''
    Computes basic stats of the histogram of an image
    '''
    img = np.array(Image.open(imPath))
    redMean, redStd, redMedian = img[:,:,0].mean(), img[:,:,0].std(), np.median(img[:,:,1])
    greenMean, greenStd, greenMedian = img[:,:,1].mean(), img[:,:,1].std(), np.median(img[:,:,1])
    blueMean, blueStd, blueMedian = img[:,:,2].mean(), img[:,:,2].std(), np.median(img[:,:,2])
    
    return [redMean, redStd, redMedian, greenMean, greenStd, greenMedian, blueMean, blueStd, blueMedian]

In [20]:
# Dull razor filter, removes hairs from pictures.
# https://www.kaggle.com/code/antocad/siim-isic-image-preprocessing-dull-razor/notebook
def dullrazor(img, lowbound=5, filterstruc=7, inpaintmat=5):

    '''
    Erases the hair from a skin image
    '''
    #grayscale
    imgtmp1 = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    #applying a blackhat
    filterSize =(filterstruc, filterstruc)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize) 
    imgtmp2 = cv2.morphologyEx(imgtmp1, cv2.MORPH_BLACKHAT, kernel)

    #0=skin and 255=hair
    ret, mask = cv2.threshold(imgtmp2, lowbound, 255, cv2.THRESH_BINARY)
    
    #inpainting
    img_final = cv2.inpaint(img, mask, inpaintmat ,cv2.INPAINT_TELEA)

    return img_final


def AdaptiveTresholding(imgName, train = True, averageFilterSize = 0):
    '''
    Computes the segmentation of an image, given its associated ID and folder
    imgName : str, id of the considered image
    train : boolean, ables to tell if we are looking in the train or test set
    showRazor : boolean, shows the step of the dullRazor algorithm is set to True
    showLuminance : boolean, shows an image of the processed luminance if set to True
    averageFilterSize : int, size of the averaging filter kernel if different than 0
    compareSeg : prompts the comparison of the calculated and the true segmentation if set to True
    '''
    
    if train:
        imgPil = Image.open(IMAGE_DIR_TRAIN + '/' + imgName +'.jpg')
    else:
        imgPil = Image.open(IMAGE_DIR_TEST + '/' + imgName +'.jpg')
        
    arrayImg = np.asarray(imgPil)
        
    
    # Step 1 : extract luminance, first channel of YIQ
    luminance = rgb2yiq(dullrazor(arrayImg))[:,:,0]
        
    # Step 2 and 3 : apply transformations and denoise
    processedLuminance = invertPixelsMinusMean(luminance)
    processedLuminance = np.uint8(processedLuminance)
    processedLuminance = cv2.fastNlMeansDenoising(processedLuminance)
    averaged = averageFilter(processedLuminance, averageFilterSize)

        
    # bwareafilt extracts the largest white area of binary img
    # Step 4 : Compute the binary image
    binaryImage = bwareafilt(averaged > averaged.std() / 2, area_range = (0, averaged.shape[0]*averaged.shape[1]/2)) # Chooses the largest segmented part
    
    
    # If the image border are black : new heuristically chosen segmentation method. The picture is first cropped then converted to black and white,
    # The distance from the mean of the new image is computed and its edges are calculated. An averaging filter is then applied and the image is inverted before thresholding.
    if (binaryImage[0:15,:]).all() and (binaryImage[:,0:15]).all() and (binaryImage[-16:-1,:]).all() and (binaryImage[:,-16:-1]).all():
        
        i=15
        while((binaryImage[i,:]).all()):
            i+=1
        j=15
        while((binaryImage[:,j]).all()):
            j+=1
            
        if train:
            imgPil = Image.open(IMAGE_DIR_TRAIN + '/' + imgName +'.jpg')
        else:
            imgPil = Image.open(IMAGE_DIR_TEST + '/' + imgName +'.jpg')

            
        image = np.array(imgPil)
        grayscale = color.rgb2gray(image)
        grayscale = grayscale[i:grayscale.shape[0]-i,j:grayscale.shape[1]-j]
        
        edge = sobel(grayscale)

        meann = grayscale.mean()
        processedImg = averageFilter(edge, 20)

        processedImg = meann - abs(grayscale - meann)
        processedImg = averageFilter(processedImg, 10)
        processedImg = 1 - processedImg
        
        binaryImage = bwareafilt(processedImg > processedImg.mean(), area_range = (0, binaryImage.shape[0]*binaryImage.shape[1]/2))

        
    # Step 5 : filling holes
    des = cv2.bitwise_not(binaryImage*255)
    contour,hier = cv2.findContours(binaryImage*255,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contour:
        cv2.drawContours(des,[cnt],0,255,-1)
    
    return des 



def getProps(anomalyName, train = True, seg = False):
    '''
    Returns the geometrical properties of an anomaly, given its associated ID and folder
    anomalyName : str, id of the lesion
    train : boolean, tells whether we are looking in the train or test set
    seg : boolean, the segmentation performed by clinicians is selected when it exists, which is told with seg
    '''
    if seg:
        if train:
            smoothImg = np.asarray(Image.open(IMAGE_DIR_TRAIN + '/' + anomalyName + '_seg' +'.png'))
        else:
            smoothImg = np.asarray(Image.open(IMAGE_DIR_TEST + '/' + anomalyName + '_seg' +'.png'))

    else:
        segImg = AdaptiveTresholding(anomalyName, train = train, averageFilterSize = 5)
        smoothImg = uniform_filter(segImg, size = 3)
        
        
    # Extract geometrical properties of binary image using regionprops_table function
    label_img = label(smoothImg)
    properties = regionprops_table(label_img, properties=('area',
    'axis_major_length',
    'axis_minor_length',
    'eccentricity',
    'equivalent_diameter_area',
    'extent',
    'feret_diameter_max',
    'inertia_tensor',
    'moments_hu',
    'perimeter'))
    
    return pd.DataFrame(properties).iloc[0]

In [21]:
def getFeatures(train = True):
    
    '''
    Returns the dataframe containing the train or test features, 
    including the true classes if it is the train features.
    '''
    
    features = []
    
    if train:
        
        train_path = IMAGE_DIR_TRAIN 
        trainFiles = [f for f in listdir(train_path) if isfile(join(train_path, f))]

        dataFrame = pd.read_csv(panda_path_train)
        
        for ID in tqdm(dataFrame.ID):

            if ((ID + '_seg.png') in trainFiles): # We take the given segmentation if it is given
                seg = True
            else:
                seg = False

            features.append(getProps(ID, True, seg))    
            
    else:
        
        test_path = IMAGE_DIR_TEST
        testFiles = [f for f in listdir(test_path) if isfile(join(test_path, f))]

        dataFrame = pd.read_csv(panda_path_test)

        for ID in tqdm(dataFrame.ID):

            if ((ID + '_seg.png') in testFiles): # We take the given segmentation if it is given
                seg = True
            else:
                seg = False

            features.append(getProps(ID, False, seg))    

            
    features = pd.DataFrame(features, columns = ['area',
    'axis_major_length',
    'axis_minor_length',
    'eccentricity',
    'equivalent_diameter_area',
    'extent',
    'feret_diameter_max',
    'inertia_tensor-0-0',      
    'inertia_tensor-0-1',
    'inertia_tensor-1-0',
    'inertia_tensor-1-1',
    'moments_hu-0',
    'moments_hu-1',
    'moments_hu-2',
    'moments_hu-3',
    'moments_hu-4',
    'moments_hu-5',
    'moments_hu-6',
    'perimeter'])

    return features

In [22]:
def getProcessedDF(train = True):
    '''
    Returns the processed dataframe, adding color stats and encoding categorical data
    '''
    colorStats = []
    
    if train:
        
        train_data = pd.read_csv(panda_path_train)
        
        # Get geometrical features
        geoTrainFeatures = getFeatures()
        
        # Get colour features
        for ID in tqdm(train_data.ID):
            colorStats.append(getRGBstats(IMAGE_DIR_TRAIN + '/' + ID + '.jpg'))
        X_train_color = pd.DataFrame(colorStats, columns = ['redMean','redStd','redMedian','greenMean','greenStd','greenMedian','blueMean','blueStd','blueMedian'])


        geoTrainFeatures.index = np.arange(len(geoTrainFeatures))
        X_train = pd.concat([train_data[['CLASS','SEX','AGE','POSITION']], geoTrainFeatures], axis = 1)
        X_train = pd.get_dummies(X_train)
        X_train = pd.concat([X_train_color,X_train], axis=1)

        # SEX_female and SEX_male are complementary, we only need one
        X_train['SEX_female'] = X_train['SEX_female'] - 0.5
        
        return X_train.drop('SEX_male', axis = 1)
        
        
    else:
        
        test_data = pd.read_csv(panda_path_test)
        
        # Get geometrical features
        geoTestFeatures = getFeatures(train = False)

        # Get colour features
        for ID in tqdm(test_data.ID):
            colorStats.append(getRGBstats(IMAGE_DIR_TEST + '/' + ID + '.jpg'))
        X_test_color = pd.DataFrame(colorStats, columns = ['redMean','redStd','redMedian','greenMean','greenStd','greenMedian','blueMean','blueStd','blueMedian'])


        geoTestFeatures.index = np.arange(len(geoTestFeatures))
        X_test = pd.concat([test_data[['SEX','AGE','POSITION']], geoTestFeatures], axis = 1)
        X_test = pd.get_dummies(X_test)
        X_test = pd.concat([X_test_color,X_test], axis=1)

        # SEX_female and SEX_male are complementary, we only need one
        X_test['SEX_female'] = X_test['SEX_female'] - 0.5
        
        return X_test.drop('SEX_male', axis = 1)


In [23]:
def getCNNstatsPreds():
    '''
    Trains a CNN from the vgg16 architecture and returns the predicted 
    probabilities for the 8 classes of the test set images.
    '''
    def preprocess(df):
        for index, img in enumerate(df.ID):
            img = img + '.jpg'
            df.ID[index] = img
        df.drop(['SEX','AGE','POSITION'], axis = 1, inplace = True)
        df.ID = df.ID.astype('string')

    def train_val_test_split(df, test_len=1000, val_ratio=0.5):
        test_rows = (np.random.rand(1000)*df.shape[0]).astype(int)
        test_df =  df.iloc[test_rows]
        test_df = test_df.reset_index().drop(['index'], axis=1)
        df.drop(test_rows, axis=0, inplace=True)
        df = df.reset_index().drop(['index'], axis=1)
        val_rows = (np.random.rand(int(val_ratio*df.shape[0]))*df.shape[0]).astype(int)
        val_df = df.iloc[val_rows]
        df.drop(val_rows, axis=0, inplace=True)
        test_df = test_df.reset_index().drop(['index'], axis=1)
        df = df.reset_index().drop(['index'], axis=1)
        return df, val_df, test_df
    
    def vgg_model(input_shape=(224,224, 3)):
        model = Sequential()
        model.add(VGG16(include_top=False, weights='imagenet', input_shape=input_shape))

        model.add(GlobalAveragePooling2D())
        #model.add(Flatten())

        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.25))

        model.add(Dense(1024))
        model.add(BatchNormalization())
        model.add(Activation('relu')) 
        model.add(Dropout(0.5))

        model.add(Dense(8, activation='sigmoid')) # Sigmoid for multiclass classification task
        model.compile(optimizer=Adam(learning_rate=1e-4), loss_weights = weights, loss='CategoricalCrossentropy', metrics=['CategoricalAccuracy'])
        print('Model has compiled')
        return model
    
    vgg16_model = vgg_model(input_shape=(224,224, 3))
    
    
    full_df = pd.read_csv(panda_path_train)
    preprocess(full_df)
    full_df = pd.get_dummies(full_df, columns =['CLASS'])
    train_df, val_df, test_df = train_val_test_split(full_df)
    labels=list(train_df.columns[1:])


    weightsamples = []

    for i in tqdm(range(len(train_df))):
        j = ((train_df.iloc[i] == 1) * 1).values.argmax()
        weightsamples.append(weights_dict[j])

    train_df = pd.concat([pd.DataFrame(weightsamples, columns=['weights']), train_df], axis = 1)


    def get_train_gen(df, img_path=IMAGE_DIR_TRAIN, target_size=(224,224)):
        data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                    horizontal_flip=True,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2)
        return data_gen.flow_from_dataframe(dataframe=df, weight_col = 'weights', directory=img_path, 
                                          x_col='ID', y_col=list(df.columns)[2:],
                                          batch_size=64, shuffle=True, class_mode='raw', 
                                          target_size=target_size)

    def get_val_test_gen(val_df, test_df, img_path=IMAGE_DIR_TRAIN, target_size=(224,224)):
        data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
        val = data_gen.flow_from_dataframe(dataframe=val_df, directory=img_path, 
                                          x_col='ID', y_col=list(val_df.columns)[1:],
                                          batch_size=64, shuffle=True, class_mode='raw', 
                                          target_size=target_size)
        test = data_gen.flow_from_dataframe(dataframe=test_df, directory=img_path, 
                                          x_col='ID', batch_size=1, shuffle=True, class_mode=None, 
                                          target_size=target_size)
        return val, test

    train_generator = get_train_gen(train_df)
    valid_generator, test_generator = get_val_test_gen(val_df, test_df)

    STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
    STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
    STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
    history = vgg16_model.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=valid_generator, validation_steps=STEP_SIZE_VALID, epochs=20)
    full_df_test = pd.read_csv(panda_path_test)

    for i in range(len(full_df_test.ID)):
        full_df_test.ID[i] = full_df_test.ID[i] + '.jpg'
    
    data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
    gen =  data_gen.flow_from_dataframe(dataframe=full_df_test,
    directory= IMAGE_DIR_TEST,
    x_col="ID",
    y_col=None,
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(224,224))
                              
    return vgg16_model.predict_generator(gen)


def getTreeStatsPreds(train_features, test_features):


    Xtrain = train_features.drop('CLASS', axis = 1)
    Xtest = test_features
        
    ytrain = train_features.CLASS - 1
    
    
    training_weights = []
    for classe in ytrain:
        training_weights.append(weights_dict[classe + 1]) # We give weights to the training samples

        
    params = {'eta' : 0.5, 'subsample' : 0.9,
    'learning_rate' : 10e-2, 'max_depth' : 5,
    'n_estimators' : 300}
    
    
    clf = xgb.XGBClassifier(eta = params['eta'], subsample = params['subsample'], learning_rate = params['learning_rate'],
                            max_depth = params['max_depth'], n_estimators = model_params['n_estimators'])
    clf.fit(Xtrain, ytrain, sample_weight = training_weights)

    return pd.DataFrame(clf.predict_proba(Xtest))

In [24]:
IMAGE_DIR_TRAIN = os.path.join('../input/ima205-challenge-2022/Train/Train')
panda_path_train = os.path.join('../input/ima205-challenge-2022/metadataTrain.csv')
IMAGE_DIR_TEST = os.path.join('../input/ima205-challenge-2022/Test/Test')
panda_path_test = os.path.join('../input/ima205-challenge-2022/metadataTest.csv')


def main(IMAGE_DIR_TRAIN = IMAGE_DIR_TRAIN, panda_path_train = panda_path_train,
         IMAGE_DIR_TEST = IMAGE_DIR_TEST, panda_path_test = panda_path_test):
    
    train_features = getProcessedDF(train = True)
    test_features = getProcessedDF(train = False)

    treeStatsPreds = getTreeStatsPreds(train_features, test_features)
    CNNstatsPreds = getCNNstatsPreds()
    
    for i in tqdm(range(6333)):
        CNNstatsPreds = CNNstatsPreds.iloc[i][[0,1,2,3,4,5,6,7]].values / CNNstatsPreds.iloc[i][[0,1,2,3,4,5,6,7]].values.sum() # CNN stats are not probabilities, we convert them
        treeStatsPreds = treeStatsPreds.iloc[i][[1,2,3,4,5,6,7,8]].values
        preds.append((0.8*CNNstatsPreds + 0.2*treeStatsPreds).argmax() + 1) # Let's give CNN_Stats more importance since it has a better accuracy*
        
    submission = pd.concat([test_features.ID, pd.DataFrame(preds)], axis = 1)
    submission.columns = ['ID', 'CLASS']
    return submission

In [ ]:
submission = main()
submission.to_csv('submission.csv', index = False)